In [1]:
import os
print(os.getcwd())
# os.chdir("..")

from src.constants import *
from src.model.my_model import MyModel
from src.model.baselines import UNet, SWINUNETR
import torch
from src.utils.metrics import *
from torch.cuda.amp import GradScaler, autocast
from monai.networks import one_hot
from src.data.data_loaders import get_loaders


/home/imreb/AAAsegmentor/notebooks


<class 'monai.transforms.utility.dictionary.AddChanneld'>: Class `AddChanneld` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead with `channel_dim='no_channel'`.
monai.transforms.utility.dictionary EnsureChannelFirstd.__init__:meta_keys: Argument `meta_keys` has been deprecated since version 0.9. not needed if image is type `MetaTensor`.
<class 'monai.transforms.utility.dictionary.AsChannelFirstd'>: Class `AsChannelFirstd` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead.


In [2]:
device = "cuda"

In [3]:
s=128
example = torch.rand(size=(1, 1, 256, 256, 64)).to(device)
mask = torch.rand(size=(4, 3, 256, 256, 64)).to(device)

example.dtype

torch.float32

In [4]:
train_loader, test_loader = get_loaders()


Loading dataset: 100%|██████████| 3/3 [00:13<00:00,  4.33s/it]


In [5]:
model = MyModel(in_channels=1,
                mid_channels=4,
                out_channels=3,
                patch_size=(4,4,4),
                embed_dim=256,
                img_size=(256, 256, 64)).to(device)

In [6]:
img = next(iter(train_loader))['img'].to(device)

In [7]:
img.shape

torch.Size([4, 1, 256, 256, 64])

In [8]:
scaler = GradScaler()

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

# with autocast():
#     out = model(img)
#     outputs = torch.softmax(out, dim=1)
#     # mask = one_hot(mask, num_classes=3)
#     loss = LOSS(outputs, mask)

# scaler.scale(loss).backward()
# scaler.step(optimizer)
# scaler.update()

train_single_epoch(model=model, optimizer=optimizer, train_loader=train_loader, scaler=scaler)

NameError: name 'train_single_epoch' is not defined

In [ ]:
out.shape

In [ ]:
import time
import os
import psutil
import gpustat
from contextlib import contextmanager

@contextmanager
def resource_monitor():
    # CPU and RAM usage monitoring
    process = psutil.Process(os.getpid())
    cpu_start = process.cpu_percent(interval=None)
    ram_start = process.memory_info().rss

    # GPU usage monitoring
    try:
        gpu_start = gpustat.new_query().jsonify()["gpus"][0]["memory.used"]
    except Exception as e:
        gpu_start = "Unable to retrieve GPU info: {}".format(e)

    # Time monitoring
    start_time = time.time()

    yield

    # Time monitoring
    elapsed_time = time.time() - start_time

    # CPU and RAM usage monitoring
    cpu_end = process.cpu_percent(interval=None)
    ram_end = process.memory_info().rss

    # GPU usage monitoring
    try:
        gpu_end = gpustat.new_query().jsonify()["gpus"][0]["memory.used"]
    except Exception as e:
        gpu_end = "Unable to retrieve GPU info: {}".format(e)

    print(f'Elapsed time: {elapsed_time} seconds')
    print(f'CPU used: {cpu_end - cpu_start} percent')
    print(f'RAM used: {ram_end - ram_start} bytes')

    if isinstance(gpu_start, float) and isinstance(gpu_end, float):
        print(f'GPU memory used: {gpu_end - gpu_start} MiB')
    else:
        print(f'GPU usage: {gpu_end}')


In [ ]:

# Usage example:
with resource_monitor():
    out = model(example) 

In [ ]:
out.shape

In [ ]:
count_learnable_parameters(model.vit.embedders) / count_learnable_parameters(model.vit.vit)

In [ ]:
import torch
from flash_attn.flash_attention import FlashMHA

# Replace this with your correct GPU device
device = "cuda:0"

# Create attention layer. This is similar to torch.nn.MultiheadAttention,
# and it includes the input and output linear layers
flash_mha = FlashMHA(
    embed_dim=256, # total channels (= num_heads * head_dim)
    num_heads=16, # number of heads
    device=device,
    dtype=torch.float16,
)

# Run forward pass with dummy data
x = torch.randn(
    (8, 512, 256), # (batch, seqlen, embed_dim)
    device=device,
    dtype=torch.float16
)

output = flash_mha(x)[0]

In [ ]:
out.shape